In [1]:
import os
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.pipeline import Pipeline

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append("..") 
from gcforest.gcforest import GCForest

import xlwt
import itertools

# 数据及参数

In [2]:
random_seed = 42
cv=5
score = 'f1_weighted'

In [3]:
def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = random_seed
    ca_config["max_layers"] = 10
    ca_config["early_stopping_rounds"] = 3
    ca_config["n_classes"] = 6
    ca_config["estimators"] = []
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "LogisticRegression"})
    config["cascade"] = ca_config
    return config

In [4]:
path = os.getcwd()+'/../data/20122018freshwater_four_feature.csv'
data = pd.read_csv(path, na_values = np.nan)

In [5]:
X = data.drop(['本周水质'], axis=1).values # Series
y = data['本周水质'].values.reshape(-1,1) - 1

# 1. 中位数填充缺失值，2.Z-score标准化
clean_pipeline = Pipeline([('imputer', preprocessing.Imputer(missing_values='NaN',strategy="median")),
                           ('std_scaler', preprocessing.StandardScaler())])
X = clean_pipeline.fit_transform(X)

In [6]:
X.shape

(33612, 4)

In [7]:
comb_3=list(itertools.combinations([0,1,2,3],3))
comb_2=list(itertools.combinations([0,1,2,3],2))

# k折交叉验证

In [8]:
# function：使用5折交叉验证统计各类别5次平均后的Acc，5次平均后的F1，和模型的总Acc以及总Weighted F1

# Input: 
#     X: 总样本
#     y: 总样本
#     model: function
#     cv: cross_validation的次数
# Output:
#     Acc_mean, 各类别的Acc
#     F1_mean, 各类别F1
#     Support_mean, 各类别预测样本占总样本的比重
#     Acc, 总Acc
#     F1_weighted 总Weighted F1
# function：使用5折交叉验证统计各类别5次平均后的Acc，5次平均后的F1，和模型的总Acc以及总Weighted F1

# Input: 
#     X: 总样本
#     y: 总样本
#     model: function
#     cv: cross_validation的次数
# Output:
#     Acc_mean, 各类别的Acc
#     F1_mean, 各类别F1
#     Support_mean, 各类别预测样本占总样本的比重
#     Acc, 总Acc
#     F1_weighted 总Weighted F1


def kftrain(X, y, model, cv):
    model_name = model.__class__.__name__

    n_samples = X.shape[0]
    n_features = X.shape[1]
    n_class = np.unique(y).shape[0]

    Acc_matrix = np.zeros((n_class, cv)) # 矩阵(6,5) 第i行：类别i的在cv组test data的cv个acc
    _Acc_matrix = np.zeros((n_class, cv)) # 矩阵(6,5) 第i行：类别i的在cv组train data的cv个acc
    F1_matrix = np.zeros((n_class, cv)) # 矩阵(6,5) 第i行：类别i的在cv组test data的cv个acc
    _F1_matrix = np.zeros((n_class, cv)) # 矩阵(6,5) 第i行：类别i的在cv组train data的cv个acc
    cv_F1_weighted = np.zeros(cv) # 5次 test data 的F1_weighted
    _cv_F1_weighted = np.zeros(cv) # 5次 train data 的F1_weighted
    cv_Acc = np.zeros(cv) # 5次test data的Acc
    _cv_Acc = np.zeros(cv) # 5次train data的Acc
    # 各类别的Support，每个类别的support由5次fold后得到的5个support求平均得到，这里support：各类别样本占总样本的比列
    # 各类别的F1，由5次fold后得到的5个F1求平均得到
    # 所以，Weighted F1 等于“5次fold得到的5个 Weighted F1 求平均得到” ——等价于—— “6个support*6个F1”得到
    Support_matrix = np.zeros((n_class, cv)) # 矩阵(6,5) 5次fold后test data各类别的support
    _Support_matrix = np.zeros((n_class, cv)) # 矩阵(6,5) 5次fold后train data各类别的support
    
    # 各类别的Mean，SD
    Acc_mean_class = np.zeros(6)
    _Acc_mean_class = np.zeros(6)
    Acc_SD_class = np.zeros(6)
    _Acc_SD_class = np.zeros(6)
    F1_mean_class = np.zeros(6)
    _F1_mean_class = np.zeros(6)
    F1_SD_class = np.zeros(6)
    _F1_SD_class = np.zeros(6)
    
    k = 0
    skf = StratifiedKFold(n_splits=cv) # 定义5折分层划分器
    
    for train_index, test_index in skf.split(X, y):
        K_train_x, K_test_x = X[train_index], X[test_index]
        K_train_y, K_test_y = y[train_index], y[test_index]
        if model_name == 'GCForest':
            model.fit_transform(K_train_x, K_train_y.reshape(K_train_y.shape[0]))
        else:
            model.fit(K_train_x, K_train_y)
    
        K_test_y_pred = model.predict(K_test_x)
        K_train_y_pred = model.predict(K_train_x)
        
        # 由混淆矩阵计算各类别的Acc
        test_cm = confusion_matrix(K_test_y, K_test_y_pred)
        train_cm = confusion_matrix(K_train_y, K_train_y_pred)
        test_acc_all_class = np.zeros(n_class) # 6个类别上的测试acc
        train_acc_all_class = np.zeros(n_class) # 6个类别上的训练acc
        
        i = 0
        for c in test_cm:
            test_acc_all_class[i] = c[i]/np.sum(c)
            i += 1
        Acc_matrix[:,k] = test_acc_all_class
        
        i = 0
        for c in train_cm:
            train_acc_all_class[i] = c[i]/np.sum(c)
            i += 1
        _Acc_matrix[:,k] = train_acc_all_class
        
        # 由classification_report提取各类别的F1
        cr = classification_report(K_test_y, K_test_y_pred, digits=4)
        _cr = classification_report(K_train_y, K_train_y_pred, digits=4)
        test_f1_all_class = np.zeros(n_class) # test data 6个类别上的测试f1
        train_f1_all_class = np.zeros(n_class) # train data 6个类别上的训练f1
        support_all_class = np.zeros(n_class) # test data 6个类别的support
        _support_all_class = np.zeros(n_class) # train data 6个类别的support
        
        i = 0
        for l in range(2,8):
            test_f1_all_class[i] = float(cr.splitlines()[l].split()[3])
            train_f1_all_class[i] = float(_cr.splitlines()[l].split()[3])
            support_all_class[i] = float(cr.splitlines()[l].split()[4])/(n_samples/cv)
            _support_all_class[i] = float(_cr.splitlines()[l].split()[4])/(n_samples*(1-1/cv))
            i = i + 1
        F1_matrix[:,k] = test_f1_all_class
        _F1_matrix[:,k] = train_f1_all_class
        Support_matrix[:,k] = support_all_class
        _Support_matrix[:,k] = _support_all_class
        
        # cv_F1_weighted[k] = float(cr.splitlines()[9].split()[5]) # 第k折时的Weighted F1
        cv_F1_weighted[k] = f1_score(K_test_y, K_test_y_pred, average="weighted")
        _cv_F1_weighted[k] = f1_score(K_train_y, K_train_y_pred, average="weighted")
        cv_Acc[k] = accuracy_score(K_test_y, K_test_y_pred)
        _cv_Acc[k] = accuracy_score(K_train_y, K_train_y_pred)
        
        k += 1
    
    Acc_mean = np.mean(Acc_matrix, axis=1) # 6个类别的k-fold平均acc
    _Acc_mean = np.mean(_Acc_matrix, axis=1) # 6个类别的k-fold平均acc
    
    F1_mean = np.mean(F1_matrix, axis=1) # 6个类别的k-fold平均F1
    _F1_mean = np.mean(_F1_matrix, axis=1) # 6个类别的k-fold平均F1
    Support_mean = np.mean(Support_matrix, axis=1) # 6个类别的k-fold平均support
    _Support_mean = np.mean(_Support_matrix, axis=1) # 6个类别的k-fold平均support
    
    Acc = np.mean(Acc_mean)
    _Acc = np.mean(_Acc_mean)
    F1_weighted = np.mean(cv_F1_weighted) # 5个Weighted F1 求平均，并可以证明其等价于 np.sum(F1_mean*Support_mean)
    _F1_weighted = np.mean(_cv_F1_weighted) # 5个Weighted F1 求平均，并可以证明其等价于 np.sum(F1_mean*Support_mean)
    SD_Acc = np.std(cv_Acc)
    _SD_Acc = np.std(cv_Acc)
    SD_F1 = np.std(cv_F1_weighted)
    _SD_F1 = np.std(cv_F1_weighted)
    
    Acc_mean_class = np.mean(Acc_matrix, axis=1)
    _Acc_mean_class = np.mean(_Acc_matrix, axis=1)
    Acc_SD_class = np.std(Acc_matrix, axis=1)
    _Acc_SD_class = np.std(_Acc_matrix, axis=1)
    F1_mean_class = np.mean(F1_matrix, axis=1)
    _F1_mean_class = np.mean(_F1_matrix, axis=1)
    F1_SD_class = np.std(F1_matrix, axis=1)
    _F1_SD_class = np.std(_F1_matrix, axis=1)
    
    # return Acc_mean, F1_mean, Support_mean, Acc, F1_weighted, SD_Acc, SD_F1, Acc_mean_class, Acc_SD_class, F1_mean_class, F1_SD_class
    return Acc_matrix, F1_matrix, Support_matrix, _Acc_matrix, _F1_matrix, _Support_matrix, Acc_mean, _Acc_mean, F1_mean, _F1_mean, Acc, _Acc, F1_weighted, _F1_weighted, SD_Acc, _SD_Acc, SD_F1, _SD_F1, Acc_mean_class, _Acc_mean_class, Acc_SD_class, _Acc_SD_class, F1_mean_class, _F1_mean_class, F1_SD_class, _F1_SD_class

In [ ]:
config = get_toy_config()

model = GCForest(config)

for cols in comb_3:
    print("DCF"+str(cols))
    X_new = X[:,cols]
    workbook = xlwt.Workbook(encoding = 'utf-8')  

    model_name = model.__class__.__name__
    print(model_name)
    Acc_matrix, F1_matrix, Support_matrix, _Acc_matrix, _F1_matrix, _Support_matrix, Acc_class, _Acc_class, F1_class, _F1_class, Acc, _Acc, F1_weighted, _F1_weighted, SD_Acc, _SD_Acc, SD_F1, _SD_F1, Acc_mean_class, _Acc_mean_class, Acc_SD_class, _Acc_SD_class, F1_mean_class, _F1_mean_class, F1_SD_class, _F1_SD_class = kftrain(X_new, y, model, cv)

    worksheet = workbook.add_sheet(model_name, cell_overwrite_ok=True)
    worksheet.write(0,0, "train_Acc_matrix")
    for i in range(6):
        for j in range(cv):
            worksheet.write(i+1, j, _Acc_matrix[i,j])

    worksheet.write(0,6, "test_Acc_matrix")
    for i in range(6):
        for j in range(cv):
            worksheet.write(i+1, j+cv+2, Acc_matrix[i,j])



    worksheet.write(11,0, "train_F1_matrix")
    for i in range(6):
        for j in range(cv):
            worksheet.write(i+12, j, _F1_matrix[i,j])

    worksheet.write(11,6, "test_F1_matrix")
    for i in range(6):
        for j in range(cv):
            worksheet.write(i+12, j+cv+2, F1_matrix[i,j])



    worksheet.write(20,0, "train_support_matrix")
    for i in range(6):
        for j in range(cv):
            worksheet.write(i+21, j, _Support_matrix[i,j])

    worksheet.write(20,6, "test_support_matrix")
    for i in range(6):
        for j in range(cv):
            worksheet.write(i+21, j+cv+2, Support_matrix[i,j])

    workbook.save("../res/DCF"+str(cols)+".xls")

[ 2019-06-03 22:30:59,538][cascade_classifier.fit_transform] X_groups_train.shape=[(26886, 3)],y_train.shape=(26886,),X_groups_test.shape=no_test,y_test.shape=no_test
[ 2019-06-03 22:30:59,539][cascade_classifier.fit_transform] group_dims=[3]
[ 2019-06-03 22:30:59,543][cascade_classifier.fit_transform] group_starts=[0]
[ 2019-06-03 22:30:59,545][cascade_classifier.fit_transform] group_ends=[3]
[ 2019-06-03 22:30:59,547][cascade_classifier.fit_transform] X_train.shape=(26886, 3),X_test.shape=(0, 3)
[ 2019-06-03 22:30:59,552][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(26886, 3), X_cur_test.shape=(0, 3)


DCF(0, 1, 2)
GCForest


/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
[ 2019-06-03 22:30:59,967][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_0.predict)=82.53%
[ 2019-06-03 22:31:00,188][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_1.predict)=83.13%
[ 2019-06-03 22:31:00,398][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_2.predict)=83.13%
[ 2019-06-03 22:31:00,625][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_3.predict)=82.49%
[ 2019-06-03 22:31:00,841][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_4.predict)=83.65%
[ 2019-06-03 22:31:00,848][kfold_wrapper.log_eval_metrics] Weighted F1 (l

[ 2019-06-03 22:31:11,358][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_1 - 5_folds.train_4.predict)=83.46%
[ 2019-06-03 22:31:11,366][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_1 - 5_folds.train_cv.predict)=83.62%
[ 2019-06-03 22:31:11,930][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_0.predict)=83.19%
[ 2019-06-03 22:31:12,452][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_1.predict)=83.18%
[ 2019-06-03 22:31:12,978][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_2.predict)=84.72%
[ 2019-06-03 22:31:13,533][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_3.predict)=83.57%
[ 2019-06-03 22:31:14,066][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_4.predict)=84.17%
[ 2019-06-03 22:31:14,074][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds

[ 2019-06-03 22:31:43,587][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_0.predict)=75.71%
[ 2019-06-03 22:31:44,109][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_1.predict)=76.34%
[ 2019-06-03 22:31:44,610][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_2.predict)=76.08%
[ 2019-06-03 22:31:45,122][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_3.predict)=75.91%
[ 2019-06-03 22:31:45,634][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_4.predict)=76.09%
[ 2019-06-03 22:31:45,642][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_cv.predict)=76.03%
[ 2019-06-03 22:31:46,186][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_0.predict)=75.37%
[ 2019-06-03 22:31:46,710][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds

...

[ 2019-06-03 23:01:51,521][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_3 - 5_folds.train_0.predict)=98.43%
[ 2019-06-03 23:01:51,801][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_3 - 5_folds.train_1.predict)=98.49%
[ 2019-06-03 23:01:52,086][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_3 - 5_folds.train_2.predict)=98.12%
[ 2019-06-03 23:01:52,371][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_3 - 5_folds.train_3.predict)=97.97%
[ 2019-06-03 23:01:52,645][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_3 - 5_folds.train_4.predict)=98.09%
[ 2019-06-03 23:01:52,652][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_3 - 5_folds.train_cv.predict)=98.22%
[ 2019-06-03 23:01:52,947][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_4 - 5_folds.train_0.predict)=96.39%
[ 2019-06-03 23:01:53,225][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_4 - 5_folds

[ 2019-06-03 23:02:05,301][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_3 - 5_folds.train_0.predict)=98.42%
[ 2019-06-03 23:02:05,417][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_3 - 5_folds.train_1.predict)=98.00%
[ 2019-06-03 23:02:05,538][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_3 - 5_folds.train_2.predict)=98.43%
[ 2019-06-03 23:02:05,654][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_3 - 5_folds.train_3.predict)=98.21%
[ 2019-06-03 23:02:05,773][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_3 - 5_folds.train_4.predict)=98.33%
[ 2019-06-03 23:02:05,780][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_3 - 5_folds.train_cv.predict)=98.28%
[ 2019-06-03 23:02:05,830][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_4 - 5_folds.train_0.predict)=97.02%
[ 2019-06-03 23:02:05,867][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_4 - 5_folds

[ 2019-06-03 23:02:15,645][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_5 - 5_folds.train_0.predict)=96.87%
[ 2019-06-03 23:02:16,034][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_5 - 5_folds.train_1.predict)=97.37%
[ 2019-06-03 23:02:16,362][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_5 - 5_folds.train_2.predict)=96.87%
[ 2019-06-03 23:02:16,678][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_5 - 5_folds.train_3.predict)=97.05%
[ 2019-06-03 23:02:17,031][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_5 - 5_folds.train_4.predict)=96.80%
[ 2019-06-03 23:02:17,039][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_5 - 5_folds.train_cv.predict)=96.99%
[ 2019-06-03 23:02:17,417][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_6 - 5_folds.train_0.predict)=96.87%
[ 2019-06-03 23:02:17,742][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_6 - 5_folds

[ 2019-06-03 23:02:38,416][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_7 - 5_folds.train_2.predict)=98.35%
[ 2019-06-03 23:02:39,232][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_7 - 5_folds.train_3.predict)=98.51%
[ 2019-06-03 23:02:40,072][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_7 - 5_folds.train_4.predict)=98.19%
[ 2019-06-03 23:02:40,080][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_7 - 5_folds.train_cv.predict)=98.36%
[ 2019-06-03 23:02:40,090][cascade_classifier.calc_f1] Weighted F1 (layer_2 - train.classifier_average)=98.34%
[ 2019-06-03 23:02:40,098][cascade_classifier.fit_transform] [layer=3] look_indexs=[0], X_cur_train.shape=(26892, 51), X_cur_test.shape=(0, 51)
[ 2019-06-03 23:02:40,466][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_0 - 5_folds.train_0.predict)=98.42%
[ 2019-06-03 23:02:40,795][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_0 - 5_folds.

[ 2019-06-03 23:02:57,135][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_1 - 5_folds.train_2.predict)=98.57%
[ 2019-06-03 23:02:57,427][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_1 - 5_folds.train_3.predict)=98.46%
[ 2019-06-03 23:02:57,712][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_1 - 5_folds.train_4.predict)=98.10%
[ 2019-06-03 23:02:57,719][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_1 - 5_folds.train_cv.predict)=98.33%
[ 2019-06-03 23:02:58,038][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_2 - 5_folds.train_0.predict)=98.38%
[ 2019-06-03 23:02:58,357][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_2 - 5_folds.train_1.predict)=98.39%
[ 2019-06-03 23:02:58,637][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_2 - 5_folds.train_2.predict)=98.34%
[ 2019-06-03 23:02:58,951][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_2 - 5_folds

In [ ]:
config = get_toy_config()

model = GCForest(config)

for cols in comb_2:
    print("DCF"+str(cols))
    X_new = X[:,cols]
    workbook = xlwt.Workbook(encoding = 'utf-8')  

    model_name = model.__class__.__name__
    print(model_name)
    Acc_matrix, F1_matrix, Support_matrix, _Acc_matrix, _F1_matrix, _Support_matrix, Acc_class, _Acc_class, F1_class, _F1_class, Acc, _Acc, F1_weighted, _F1_weighted, SD_Acc, _SD_Acc, SD_F1, _SD_F1, Acc_mean_class, _Acc_mean_class, Acc_SD_class, _Acc_SD_class, F1_mean_class, _F1_mean_class, F1_SD_class, _F1_SD_class = kftrain(X_new, y, model, cv)

    worksheet = workbook.add_sheet(model_name, cell_overwrite_ok=True)
    worksheet.write(0,0, "train_Acc_matrix")
    for i in range(6):
        for j in range(cv):
            worksheet.write(i+1, j, _Acc_matrix[i,j])

    worksheet.write(0,6, "test_Acc_matrix")
    for i in range(6):
        for j in range(cv):
            worksheet.write(i+1, j+cv+2, Acc_matrix[i,j])



    worksheet.write(11,0, "train_F1_matrix")
    for i in range(6):
        for j in range(cv):
            worksheet.write(i+12, j, _F1_matrix[i,j])

    worksheet.write(11,6, "test_F1_matrix")
    for i in range(6):
        for j in range(cv):
            worksheet.write(i+12, j+cv+2, F1_matrix[i,j])



    worksheet.write(20,0, "train_support_matrix")
    for i in range(6):
        for j in range(cv):
            worksheet.write(i+21, j, _Support_matrix[i,j])

    worksheet.write(20,6, "test_support_matrix")
    for i in range(6):
        for j in range(cv):
            worksheet.write(i+21, j+cv+2, Support_matrix[i,j])

    workbook.save("../res/DCF"+str(cols)+".xls")

[ 2019-06-03 23:03:10,948][cascade_classifier.fit_transform] X_groups_train.shape=[(26886, 2)],y_train.shape=(26886,),X_groups_test.shape=no_test,y_test.shape=no_test
[ 2019-06-03 23:03:10,949][cascade_classifier.fit_transform] group_dims=[2]
[ 2019-06-03 23:03:10,950][cascade_classifier.fit_transform] group_starts=[0]
[ 2019-06-03 23:03:10,951][cascade_classifier.fit_transform] group_ends=[2]
[ 2019-06-03 23:03:10,952][cascade_classifier.fit_transform] X_train.shape=(26886, 2),X_test.shape=(0, 2)
[ 2019-06-03 23:03:10,955][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(26886, 2), X_cur_test.shape=(0, 2)


DCF(0, 1)
GCForest


[ 2019-06-03 23:03:11,204][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_0.predict)=43.72%
[ 2019-06-03 23:03:11,431][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_1.predict)=42.89%
[ 2019-06-03 23:03:11,658][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_2.predict)=44.33%
[ 2019-06-03 23:03:11,882][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_3.predict)=43.73%
[ 2019-06-03 23:03:12,107][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_4.predict)=42.75%
[ 2019-06-03 23:03:12,114][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_cv.predict)=43.51%
[ 2019-06-03 23:03:12,356][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_1 - 5_folds.train_0.predict)=44.24%
[ 2019-06-03 23:03:12,583][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_1 - 5_folds

[ 2019-06-03 23:03:23,134][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_0.predict)=46.99%
[ 2019-06-03 23:03:23,771][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_1.predict)=46.95%
[ 2019-06-03 23:03:24,360][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_2.predict)=48.50%
[ 2019-06-03 23:03:24,944][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_3.predict)=48.02%
[ 2019-06-03 23:03:25,516][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_4.predict)=47.69%
[ 2019-06-03 23:03:25,523][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_cv.predict)=47.64%
[ 2019-06-03 23:03:26,110][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds.train_0.predict)=48.27%
[ 2019-06-03 23:03:26,682][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds

[ 2019-06-03 23:03:56,685][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_0.predict)=39.85%
[ 2019-06-03 23:03:57,145][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_1.predict)=39.58%
[ 2019-06-03 23:03:57,610][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_2.predict)=39.20%
[ 2019-06-03 23:03:58,067][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_3.predict)=39.75%
[ 2019-06-03 23:03:58,534][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_4.predict)=39.82%
[ 2019-06-03 23:03:58,541][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_cv.predict)=39.64%
[ 2019-06-03 23:03:59,018][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_0.predict)=40.01%
[ 2019-06-03 23:03:59,486][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds

...

[ 2019-06-03 23:48:01,303][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_3 - 5_folds.train_cv.predict)=65.02%
[ 2019-06-03 23:48:01,777][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_4 - 5_folds.train_0.predict)=55.13%
[ 2019-06-03 23:48:02,240][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_4 - 5_folds.train_1.predict)=54.12%
[ 2019-06-03 23:48:02,710][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_4 - 5_folds.train_2.predict)=55.61%
[ 2019-06-03 23:48:03,195][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_4 - 5_folds.train_3.predict)=55.23%
[ 2019-06-03 23:48:03,675][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_4 - 5_folds.train_4.predict)=55.14%
[ 2019-06-03 23:48:03,682][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_4 - 5_folds.train_cv.predict)=55.05%
[ 2019-06-03 23:48:04,170][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_5 - 5_fold

DCF(2, 3)
GCForest


[ 2019-06-03 23:48:18,855][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_1.predict)=86.59%
[ 2019-06-03 23:48:18,978][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_2.predict)=85.98%
[ 2019-06-03 23:48:19,102][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_3.predict)=86.34%
[ 2019-06-03 23:48:19,226][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_4.predict)=86.05%
[ 2019-06-03 23:48:19,234][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_0 - 5_folds.train_cv.predict)=86.21%
[ 2019-06-03 23:48:19,385][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_1 - 5_folds.train_0.predict)=85.66%
[ 2019-06-03 23:48:19,516][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_1 - 5_folds.train_1.predict)=85.96%
[ 2019-06-03 23:48:19,641][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_0 - estimator_1 - 5_folds

[ 2019-06-03 23:48:28,989][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_1.predict)=87.95%
[ 2019-06-03 23:48:29,551][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_2.predict)=87.44%
[ 2019-06-03 23:48:30,124][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_3.predict)=88.23%
[ 2019-06-03 23:48:30,691][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_4.predict)=88.35%
[ 2019-06-03 23:48:30,698][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_2 - 5_folds.train_cv.predict)=87.95%
[ 2019-06-03 23:48:31,298][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds.train_0.predict)=88.40%
[ 2019-06-03 23:48:31,855][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds.train_1.predict)=88.44%
[ 2019-06-03 23:48:32,419][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_1 - estimator_3 - 5_folds

[ 2019-06-03 23:48:57,833][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_3.predict)=80.40%
[ 2019-06-03 23:48:58,232][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_4.predict)=80.60%
[ 2019-06-03 23:48:58,240][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_4 - 5_folds.train_cv.predict)=80.31%
[ 2019-06-03 23:48:58,659][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_0.predict)=80.33%
[ 2019-06-03 23:48:59,029][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_1.predict)=80.58%
[ 2019-06-03 23:48:59,444][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_2.predict)=80.70%
[ 2019-06-03 23:48:59,830][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds.train_3.predict)=80.83%
[ 2019-06-03 23:49:00,217][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_5 - 5_folds